In [1]:
%reset -f
import sys
sys.path.insert(0,'..')
# import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import utils
from wavhandler import *
from utils_train import train_test_val_split

In [2]:
data1 = Dataset('Leafminers')
data1.read(data=data1.target_classes[0], setting='psd_dB', labels='text')
data2 = Dataset('LG')
data2.read(data=data2.target_classes[0], setting='psd_dB', labels='text')
data3 = Dataset('LG')
data3.read(data=data3.target_classes[1], setting='psd_dB', labels='text')
data4 = Dataset('Pcfruit')
data4.read(data=data4.target_classes[1], setting='psd_dB', labels='text')

Data: Cichorii.
Read 1451 filenames in 0.04 seconds.
Loaded data into matrix in 0.50 seconds.
Data: LG_drosophila_10_09.
Read 5536 filenames in 0.09 seconds.
Loaded data into matrix in 1.35 seconds.
Data: LG_zapr_26_09.
Read 7210 filenames in 0.08 seconds.
Loaded data into matrix in 1.82 seconds.
Data: D. suzukii.
Read 2401 filenames in 0.02 seconds.
Loaded data into matrix in 0.65 seconds.


In [3]:
data1.clean(plot=False)
data2.clean(plot=False)
data3.clean(plot=False)
data4.clean(plot=False)

data = pd.DataFrame()
data['fnames'] = pd.concat([data1.filenames, data2.filenames, data3.filenames, data4.filenames], axis=0).reset_index(drop=True)
data['y'] = data.fnames.apply(lambda x: x.split('/')[6])
# big.dropna(how='any', axis=1, inplace=True)
data.sample(10)

753 filenames after cleaning.
5277 filenames after cleaning.
7002 filenames after cleaning.
2373 filenames after cleaning.


,fnames,y
12791,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
7383,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
9661,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
4415,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
10149,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
4736,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
1844,/home/kalfasyan/data/insects/LG/LG_drosophila_...,LG_drosophila_10_09
7223,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
12033,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09
12569,/home/kalfasyan/data/insects/LG/LG_zapr_26_09/...,LG_zapr_26_09


In [4]:
X,y = data.fnames, data.y

In [5]:
# X,y = data.filenames.values.reshape(-1,1), data.y.values

model_name='first_' 
setting='raw'

In [6]:
top_weights_path = TEMP_DATADIR + str(model_name) + '.h5'
targets = len(np.unique(y))

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

if setting == 'psd_dB':
    X = transform_data(X, setting=setting)

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X.tolist(),y.tolist(), random_state=0)



## MODEL

In [10]:
from tensorflow.keras import Input, Model
# Build the Neural Network
model = Sequential()

inputs = Input(shape=(5000, 1))
x = Conv1D(16, 3, activation='relu')(inputs)
x = Conv1D(16, 3, activation='relu')(x)
x = BatchNormalization()(x)

x = Conv1D(32, 3, activation='relu')(x)
x = Conv1D(32, 3, activation='relu')(x)
x = BatchNormalization()(x)

x = Conv1D(64, 3, activation='relu')(x)
x = Conv1D(64, 3, activation='relu')(x)
x = BatchNormalization()(x)

x = Conv1D(128, 3, activation='relu')(x)
x = Conv1D(128, 3, activation='relu')(x)
x = BatchNormalization()(x)

x = Conv1D(256, 3, activation='relu')(x)
x = Conv1D(256, 3, activation='relu')(x)
x = BatchNormalization()(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.5)(x, training=True)

outputs = Dense(targets, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_accuracy', verbose = 1, save_best_only = True, save_weights_only = True),
    EarlyStopping(monitor = 'val_accuracy', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]


In [11]:
import tensorflow
import tensorflow.keras.backend as K
tensorflow.compat.v1.disable_eager_execution()

# for some model with dropout ...
f = K.function([model.layers[0].input, K.learning_phase()],[model.layers[-1].output])


### Fit

In [ ]:
import math
from utils_train import train_generator, valid_generator
target_names = list(np.unique(data.y))
batch_size = 32
model.fit_generator(train_generator(X_train, y_train, batch_size=batch_size,
                                   target_names=target_names,
                                   setting=setting),
                   steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
                   epochs=100,
                   validation_data = valid_generator(X_val, y_val,
                                                    batch_size=batch_size,
                                                    target_names=target_names,
                                                    setting=setting),
                    validation_steps=int(math.ceil(float(len(X_test))/float(batch_size))),
                    callbacks = callbacks_list)

## Test model

In [12]:
top_weights_path

'/media/kalfasyan/HGST_4TB/Ubudirs/projects/wingbeat_frequencies/Jupyter_Notebooks/temp_data/first_.h5'

In [16]:
model.load_weights(top_weights_path)

your_data = make_df_parallel(names=X_test, setting='raw')
your_data = np.float32(your_data.values).reshape((1541,5000,1))

your_data = your_data[:400,:,:]
your_data.shape

(400, 5000, 1)

In [17]:
n_iter = 100
no_classes = 4
x = your_data

result = np.zeros((n_iter,) + (x.shape[0], no_classes) )
for i in range(n_iter):
    result[i, :, :] = f((x,1))[0]
    
prediction = result.mean(axis=0)
uncertainty = result.std(axis=0)

In [40]:
from sklearn.metrics import accuracy_score
print( accuracy_score(prediction.argmax(axis=1), np.array(y_test)[:400]) )
uncertainty[prediction.argmax(axis=1)].max()

0.9875


0.006690433972530661

In [44]:
dataa = Dataset('Leafminers')
dataa.read(data=data1.target_classes[1], setting='raw', labels='text')

Data: Cichorii_v2.
Read 3900 filenames in 0.04 seconds.
Loaded data into matrix in 0.42 seconds.


In [55]:
your_data2 = np.float32(dataa.X.values).reshape((dataa.X.shape[0],5120,1))
your_data2 = your_data2[:100, :5000, :]
your_data2.shape

(100, 5000, 1)

In [60]:
n_iter = 100
no_classes = 4
x = your_data2

result = np.zeros((n_iter,) + (x.shape[0], no_classes) )
for i in range(n_iter):
    result[i, :, :] = f((x,1))[0]
    
prediction = result.mean(axis=0)
uncertainty = result.std(axis=0)

In [65]:
from sklearn.metrics import accuracy_score
# print( accuracy_score(prediction.argmax(axis=1), np.array(your_data2)[:100]) )
# uncertainty[prediction.argmax(axis=1)].mean()
prediction.argmax(axis=1)

array([0, 2, 1, 3, 2, 1, 2, 1, 3, 2, 3, 2, 3, 2, 1, 2, 1, 1, 1, 2, 3, 3,
       2, 1, 3, 3, 1, 2, 3, 3, 3, 2, 2, 2, 2, 1, 1, 3, 3, 0, 1, 2, 1, 1,
       0, 2, 1, 2, 1, 1, 3, 2, 3, 3, 1, 0, 3, 3, 2, 1, 0, 2, 3, 2, 2, 3,
       3, 1, 3, 3, 2, 3, 1, 1, 3, 2, 3, 1, 2, 2, 3, 3, 1, 3, 1, 3, 1, 3,
       0, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 3])

In [ ]:
# def predict_with_uncertainty(f, x, no_classes, n_iter=100):
#     result = np.zeros((n_iter,) + (x.shape[0], no_classes) )

#     for i in range(n_iter):
#         result[i,:, :] = f((x, 1))[0]

#     prediction = result.mean(axis=0)
#     uncertainty = result.std(axis=0)
#     return prediction, uncertainty    
# # predictions, uncertainty = 
# predict_with_uncertainty(model,your_data, no_classes=4, n_iter=10)

In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate_generator(valid_generator(X_test, 
                                                    y_test, 
                                                    batch_size=batch_size, 
                                                    setting=setting, 
                                                    target_names=target_names),
        steps = int(math.ceil(float(len(X_test)) / float(batch_size))))

print('loss', loss)
print('Test accuracy:', acc)

from tensorflow.keras.models import model_from_yaml
# serialize model to YAML
model_yaml = model.to_yaml()
with open(TEMP_DATADIR + model_name + "_raw_final.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)